# Testing code for OLU project UROP Spring of 2017

Aditya Pidaparti, Arindam Banerjee, Nicholas Johnson

## Data Setup
The following cell contains all code that will be become datasetup.py.

In [6]:
from pandas_datareader.data import DataReader as web #irrelevant
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

#datasets stored in the datafile, use the following to find them
#Below are the price relatives for various standard datasets
dija = '../Data/dija.csv'
msci = '../Data/msci.csv'
nyse_n = '../Data/nyse-n.csv'
nyse_o = '../Data/nyse-o.csv'
sp500 = '../Data/sp500.csv'
tse = '../Data/tse.csv'
current = msci


data = np.loadtxt(current,delimiter=',')
print (data.shape)

(1043, 24)


## Lazy Updates ADMM
The following cells contains the new lazy_updates_admm.py code

This is our main function wrapper. It's going to perform setup for each test we do. That is, once we run this function, our initial paramters have been inputted and will stay that way (i.e. the learning rate and the L1 norm). It's going to come in the form of a function return results in the form of a pandas dataframe and a dictionary with all the parameters

### Parameters we'll be varying:
* learning rate or eta
* L1 norm weight or alpha 
* transaction cost or gamma
* risk tolerance or max_vol


In [2]:
def lazy_updates_admm(PRM,betas, eta = 0.05, alpha = 0.01, gamma = 0.0 ,max_vol = 10 ** 10, debug = False):
    r = .01 #Augmentation term
    b = 2 #weight on L2 norm

    if type(prices) != np.ndarray or type (betas) != np.ndarray:
        raise Exception ('Wrong input types')
    #Make sure all incoming data is in the form of Numpy arrays or matrices.
    
    #Variable initialization
    (num_stock, num_days) = PRM.shape
    weight = np.zeros(PRM.shape)
    wealth = np.zeros((num_days, 1))
    wealth[0,0] = 1 # Start with 1 dollar on day 1
    
    #portfolio for 1st day, allocate wealth equally across assets
    weight[0:, 0] = np.ones(num_stock) * (1/num_stock)
    
    #2nd through last day
    for day in range(1, num_days):

        prev_day_weight = weight[:, day-1]
        prev_day_data = PRM[:, day-1]

        w = sparse_port_admm(prev_day_weight, prev_day_data, eta, b, alpha, r)
       
        weight[:, day] = w
        new_wealth = wealth[day-1, 0] * np.dot(weight[:, day], PRM[:, day])
        
        #Apply transaction cost. Zero for a gamma of 0
        transaction_cost = gamma*abs(wealth[day-1, 0])*np.linalg.norm(weight[:,day-1]-weight[:,day],1)
        wealth[day, 0] = new_wealth - transaction_cost

    # plt.plot(wealth[0:day])
    # plt.show()
    return [eta, alpha, gamma, rho, beta, wealth[num_days -1, 0]]

Sandbox

In [38]:
a = [[x, x+1, x+2] for x in range(1,10,3)]
print (a)
b = np.array(a)
print (type(b))

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
<class 'numpy.ndarray'>


The following cell contains the new sparse_updates_admm.py code

The following cell contains the new supporting_funcs code